In [1]:
from modelscope import snapshot_download
model_dir = snapshot_download("maple77/zpoint_large_embedding_zh")

Downloading: 100%|██████████| 172/172 [00:00<00:00, 393B/s]
Downloading: 100%|██████████| 51.0/51.0 [00:00<00:00, 121B/s]
Downloading: 100%|██████████| 341/341 [00:00<00:00, 809B/s]
Downloading: 100%|██████████| 7.01M/7.01M [00:00<00:00, 14.5MB/s]
Downloading: 100%|██████████| 1.21G/1.21G [00:03<00:00, 375MB/s] 
Downloading: 100%|██████████| 26.8k/26.8k [00:01<00:00, 21.3kB/s]
Downloading: 100%|██████████| 53.0/53.0 [00:00<00:00, 126B/s]
Downloading: 100%|██████████| 695/695 [00:00<00:00, 1.61kB/s]
Downloading: 100%|██████████| 429k/429k [00:00<00:00, 1.01MB/s]
Downloading: 100%|██████████| 1.43k/1.43k [00:00<00:00, 2.88kB/s]
Downloading: 100%|██████████| 107k/107k [00:00<00:00, 206kB/s]  


In [2]:
model_dir = snapshot_download("Xorbits/bge-reranker-base")

2024-06-25 13:55:04,378 - modelscope - WARNING - Model revision not specified, use revision: v0.0.1
Downloading: 100%|██████████| 799/799 [00:00<00:00, 1.72kB/s]
Downloading: 100%|██████████| 52.0/52.0 [00:00<00:00, 124B/s]
Downloading: 100%|██████████| 185/185 [00:00<00:00, 507B/s]
Downloading: 100%|██████████| 1.04G/1.04G [00:03<00:00, 339MB/s] 
Downloading: 100%|██████████| 174/174 [00:00<00:00, 370B/s]
Downloading: 100%|██████████| 1.55k/1.55k [00:00<00:00, 4.00kB/s]
Downloading: 100%|██████████| 1.14k/1.14k [00:00<00:00, 2.86kB/s]
Downloading: 100%|██████████| 4.83M/4.83M [00:00<00:00, 10.8MB/s]
Downloading: 100%|██████████| 167/167 [00:00<00:00, 419B/s]


In [73]:
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import tiktoken
token_count = 0
documents = PyPDFLoader("soybean_konw.pdf").load()
for i in range(len(documents)):
    encoder = tiktoken.get_encoding("gpt2") # 或其他适用的编码器
    tokens = encoder.encode(documents[0].page_content)
    token_count += len(tokens)
print(token_count)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
texts = text_splitter.split_documents(documents[:])
for idx, text in enumerate(texts):
    text.metadata["id"] = idx

13620


In [74]:
from langchain_chroma import Chroma
from langchain.schema import Document
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

top_k = 10

model_name = '/mnt/workspace/.cache/modelscope/hub/maple77/zpoint_large_embedding_zh'
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
vectorstore = Chroma.from_documents(documents=texts[:], embedding=hf, persist_directory="soybean_konw")
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": top_k}
)

In [8]:
vectorstore2 = Chroma(persist_directory="soybean_konw", embedding_function=hf)
vectorstore2._collection.get(ids=['0438ad1e-e6a7-44b2-af92-b43b8bcbb06c'])

In [38]:
from sentence_transformers import SentenceTransformer
emmbeding_model = SentenceTransformer('/mnt/workspace/.cache/modelscope/hub/maple77/zpoint_large_embedding_zh')
z = emmbeding_model.encode(texts[22].page_content, normalize_embeddings=True)
vectorstore2._collection.add('new1', z.tolist(), texts[22].metadata, texts[22].page_content)

In [ ]:
retriever.batch(['plant'])

In [70]:
reranker_args = {
    'model': '/mnt/workspace/.cache/modelscope/hub/Xorbits/bge-reranker-base',
    'top_n': 5,
    'device': 'cpu',
}

In [71]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('/mnt/workspace/.cache/modelscope/hub/Xorbits/bge-reranker-base')
rerank_model = AutoModelForSequenceClassification.from_pretrained('/mnt/workspace/.cache/modelscope/hub/Xorbits/bge-reranker-base')

In [72]:
import torch
rerank_model.eval()

pairs = [['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']]
with torch.no_grad():
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    scores = rerank_model(**inputs, return_dict=True).logits.view(-1, ).float()
    print(scores)

tensor([-8.1544,  6.1821])


In [ ]:

for query_idx in range(len(questions)):
		# 首先进行BM25检索
    doc_scores = bm25.get_scores(jieba.lcut(questions[query_idx]["question"]))
    max_score_page_idxs = doc_scores.argsort()[-3:]
		
		# top3进行重排序
    pairs = []
    for idx in max_score_page_idxs:
        pairs.append([questions[query_idx]["question"], pdf_content[idx]['content']])

    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    with torch.no_grad():
        inputs = {key: inputs[key].cuda() for key in inputs.keys()}
        scores = rerank_model(**inputs, return_dict=True).logits.view(-1, ).float()

    max_score_page_idx = max_score_page_idxs[scores.cpu().numpy().argmax()]
    questions[query_idx]['reference'] = 'page_' + str(max_score_page_idx + 1)

